In [1]:
!nvidia-smi

Fri Apr 21 21:17:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
! pip install transformers
! pip install datasets
! pip install sentencepiece
!pip install rouge_score

In [3]:
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install py7zr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.3 MB/s eta 0:00:00


In [5]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
from datasets import load_dataset, load_metric
import datasets

In [7]:
from datasets import load_dataset

def merge_splits(data):
  data = datasets.concatenate_datasets([data["train"], data["validation"], data["test"]])
  return data

data_AMI = merge_splits(load_dataset("knkarthick/AMI"))
data_samsum = merge_splits(load_dataset("samsum"))
data_dialogsum = merge_splits(load_dataset("knkarthick/dialogsum"))

# Take a look at the data
for k, v in data_AMI[0].items():
    print(k)
    print(v)
for k, v in data_samsum[0].items():
    print(k)
    print(v)
for k, v in data_dialogsum[0].items():
    print(k)
    print(v)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--AMI-e0c25640a9a75f54/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


...

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

id
30
dialogue
Speaker A: Cool. Do you wanna give me the little cable thing? Yeah. Cool. Ah, that's why it won't meet. Okay, cool. Yep, cool. Okay, functional requirements. Alright, yeah. It's working. Cool, okay. So what I have, wh where I've got my information from is a survey where the usability lab um observed remote control use with um a hundred subjects and then they gave them a questionnaire. Um so it was all about, you know, how people feel about the look and feel of the remote control, you know. What's the most annoying things about remote controls and um the possibility of speech recognition and L_C_D_ screens in remote control. Not that they actually gave me any answers on the L_C_D_ screens, so I should have taken that bit out, but anyway. Um okay, so. What they found is that people don't like how current remote controls are, so you know, definitely you should be looking at something quite different. Um seventy five percent of users find most remote controls ugly. Uh the ot

In [8]:
data_AMI

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 279
})

In [35]:
def flatten(example):
    return {
        "dialogue": example["dialogue"],
        "summary": example["summary"],
    }


data_AMI = data_AMI.map(flatten, remove_columns=["id"])
#data_samsum = data_samsum.map(flatten, remove_columns=["id"])
#data_dialogsum = data_dialogsum.map(flatten, remove_columns=["id", "topic"])

#dataset = datasets.concatenate_datasets([data_AMI, data_samsum, data_dialogsum])
dataset = data_AMI

train_data_txt, validation_data_txt = dataset.train_test_split(test_size=0.1).values()

ValueError: ignored

In [10]:
dataset

Dataset({
    features: ['dialogue', 'summary'],
    num_rows: 31108
})

In [11]:
train_data_txt[0]

{'dialogue': "David: I'm in the shop right now and I want to buy some wine, which one do you prefer?\r\nBarbara: Semisweet and white will be perfect\r\nDavid: <file_photo>\r\nBarbara: The one to the left in a green bottle :D\r\nDavid: As you wish Madam!\r\nBarbara: Make it two bottles then :D",
 'summary': 'David will buy 2 bottles of wine for Barbara.'}

In [12]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [13]:
from transformers import AutoTokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [15]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["dialogue"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch


train_data = train_data_txt.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/27997 [00:00<?, ? examples/s]

Map:   0%|          | 0/3111 [00:00<?, ? examples/s]

In [16]:
train_data.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
validation_data.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [17]:
from transformers import AutoModelForSeq2SeqLM

In [18]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [19]:
# set generate hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [20]:
rouge = load_metric("rouge")

<ipython-input-20-6f1857dc142f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


In [21]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [22]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [24]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    output_dir="led-finetuned-meetings",
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    #output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    push_to_hub=True
)

In [25]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=validation_data,
)

Cloning https://huggingface.co/jainr3/led-finetuned-meetings into local empty directory.


In [26]:
! pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=cc94d2f8f9f5594b8161774a7cb35faefa323280a84b98c43e3a58a06a02f259
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [27]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [31]:
from datetime import datetime
if WANDB_INTEGRATION:
    wandb_run = wandb.init(
        project="led_AMI",
        config={
            "per_device_train_batch_size": training_args.per_device_train_batch_size,
            "learning_rate": training_args.learning_rate,
            "dataset": "AMI",
        },
    )

    now = datetime.now()
    current_time = now.strftime("%H%M%S")
    wandb_run.name = "run" + "_" + current_time

In [32]:
# Takes way too long...
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LEDTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


KeyboardInterrupt: ignored

In [33]:
if WANDB_INTEGRATION:
    wandb_run.finish()

In [34]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/618M [00:00<?, ?B/s]

Upload file runs/Apr21_20-59-38_1400f29fc4bf/events.out.tfevents.1682111078.1400f29fc4bf.206.0:   0%|         …

Upload file training_args.bin:   0%|          | 1.00/3.68k [00:00<?, ?B/s]

Upload file runs/Apr21_20-59-38_1400f29fc4bf/1682111078.9248025/events.out.tfevents.1682111078.1400f29fc4bf.20…

To https://huggingface.co/jainr3/led-finetuned-meetings
   c8a3b44..05dd95f  main -> main

   c8a3b44..05dd95f  main -> main

To https://huggingface.co/jainr3/led-finetuned-meetings
   05dd95f..9c21fb8  main -> main

   05dd95f..9c21fb8  main -> main



'https://huggingface.co/jainr3/led-finetuned-meetings/commit/05dd95f6b04f47f8667ca93f527d9a9906af90f3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
torch.save(led.state_dict(), "/content/drive/MyDrive/led-model.pth")